In [4]:
import pandas as pd
import numpy as np
import os
from IPython.display import display

## each raw data should have three cols:
### SNP1, SNP2, STAT(strength of interaction)

In [ ]:
data = data.sort_values('STAT', ascending=False)
data[['SNP1', 'SNP2']] = np.sort(data[['SNP1', 'SNP2']].values, axis=1)
data['Interactions'] = data['SNP1'] + '-' +data['SNP2']
####################################
# to specify: 
### data_path, method_name

data.to_csv('{}/{}.csv'.format(data_path, method_name))

In [5]:
def aggregation(data_type, trait_type, method, num_pairs):
    name = '_'.join([method, data_type, trait_type])
    data = pd.read_csv(
        '/massstorage/URT/GEN/BIO3/Bowen/Epistasis/{}/{}/{}.csv'.format(data_type, trait_type, method))
    data = data['Interactions'].tolist()[:num_pairs]
    return name, data

In [9]:
all_rankings = {}
num_pairs = 1000
for data in ['filtered', 'unfiltered']:
    for trait in ['corrected', 'uncorrected']:
        path = '/massstorage/URT/GEN/BIO3/Bowen/Epistasis/{}/{}/'.format(data, trait)
        methods = os.listdir(path)
        methods = [method.split('.csv')[0] for method in methods]
        for method in methods:
            name, ranking = aggregation(data, trait, method, num_pairs)
            all_rankings[name] = ranking

concordance_table = pd.DataFrame()
intersection_interactions = {}
Graph = []
k = 0
for i in range(len(all_rankings.keys())):
    for j in range(i, len(all_rankings.keys())):
        key_a = list(all_rankings.keys())[i]
        key_b = list(all_rankings.keys())[j]
        concordance_table.loc[key_a, key_b] = int(
            len(set(all_rankings[key_a]).intersection(all_rankings[key_b])))
        if key_a != key_b:
            intersection = int(
                len(set(all_rankings[key_a]).intersection(all_rankings[key_b])))
            
            intersection_interactions['{} : {}'.format(key_a, key_b)] = intersection
            
            Graph.append([key_a, key_b, intersection])
            
            k += 1

In [10]:
Graph = pd.DataFrame(Graph)
Graph.columns = ['Node1', 'Node2', 'Strength']

In [11]:
Graph.to_csv('/massstorage/URT/GEN/BIO3/Bowen/Epistasis/GraphTop{}.csv'.format(num_pairs))

In [4]:
display(concordance_table.fillna(''))
contingency_table.to_csv('/massstorage/URT/GEN/BIO3/Bowen/Epistasis/ConcordanceTableTop{}.csv'.format(num_pairs))

,epiGTBN-ReliefF_filtered_corrected,casmap-region-gwas-rec_filtered_corrected,episcan_filtered_corrected,antEpiSeeker_filtered_corrected,NNweights_filtered_corrected,plink-linearRegression-Hector_filtered_corrected,epiHSIC_filtered_corrected,plink-LR_filtered_corrected,lightGBM_filtered_corrected,mbmdr-gammaMaxt-lowerOrder_filtered_corrected,...,lightGBM_unfiltered_corrected,mbmdr-gammaMaxt-lowerOrder_unfiltered_corrected,epiGTBN-Episcan_unfiltered_corrected,mbmdr-gammaMaxt_unfiltered_corrected,epiGTBN-ReliefF_unfiltered_uncorrected,episcan_unfiltered_uncorrected,plink-boost_unfiltered_uncorrected,NNweights_unfiltered_uncorrected,lightGBM_unfiltered_uncorrected,epiGTBN-Episcan_unfiltered_uncorrected
epiGTBN-ReliefF_filtered_corrected,100.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,10.0,1.0,3.0,4.0,0.0,0.0,0.0,9.0
casmap-region-gwas-rec_filtered_corrected,,56.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
episcan_filtered_corrected,,,100.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,1.0,0.0,0.0,0.0
antEpiSeeker_filtered_corrected,,,,88.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
NNweights_filtered_corrected,,,,,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
plink-linearRegression-Hector_filtered_corrected,,,,,,64.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
epiHSIC_filtered_corrected,,,,,,,28.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
plink-LR_filtered_corrected,,,,,,,,100.0,0.0,0.0,...,0.0,0.0,6.0,25.0,2.0,14.0,0.0,0.0,0.0,8.0
lightGBM_filtered_corrected,,,,,,,,,100.0,3.0,...,35.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,21.0,0.0
mbmdr-gammaMaxt-lowerOrder_filtered_corrected,,,,,,,,,,100.0,...,3.0,44.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0


NameError: name 'contingency_table' is not defined

In [ ]:
data_types = ['filtered', 'unfiltered']
trait_types = ['corrected', 'uncorrected']
methods = concordance_table.columns.tolist()
combinations = {}
for data_type in data_types:
    for trait_type in trait_types:
        value =  [col for col in methods if '_{}_{}'.format(data_type, trait_type) in col]
        combinations['{}_{}'.format(data_type, trait_type)] = concordance_table.loc[value, value]

for key, value in combinations.items():
    
    display(value.fillna(''))

In [ ]:

for key, value in combinations.items():
    intersection_interactions = {}
    for i in range(len(value.columns)):
        for j in range(i, len(value.columns)):
            key_a = value.columns[i]
            key_b = value.columns[j]

            if key_a != key_b:
                intersection_interactions[
                    '{} : {}'.format(key_a.split('_')[0], key_b.split('_')[0])] = value.loc[key_a, key_b]
                
    sub_concordance_table = pd.DataFrame.from_dict(intersection_interactions, 
                                               orient='index', columns=[
                                            'concordance_{}_{}'.format(key_a.split('_')[1], key_a.split('_')[2])])
    display(sub_concordance_table.sort_values(
        'concordance_{}_{}'.format(key_a.split('_')[1], key_a.split('_')[2]), ascending=False).iloc[:10])

In [ ]:
intersection_interactions = {}
for i in range(len(concordance_table.columns)):
    for j in range(i, len(concordance_table.columns)):
        key_a = concordance_table.columns[i]
        key_b = concordance_table.columns[j]

        if key_a != key_b:
            intersection_interactions['{} : {}'.format(key_a, key_b)] = concordance_table.loc[key_a, key_b]

sub_concordance_table = pd.DataFrame.from_dict(intersection_interactions, 
                                           orient='index', columns=['concordance'])
display(sub_concordance_table.sort_values('concordance', ascending=False).iloc[:20])